<h1 style='text-aling:center;color:Navy'>  Big Data Systems - Laboratory 1  </h1>

#### Topics included in this lab:
 1) Map-Reduce (20 minutes)
 2) Semistructured DS: Spanner DB (20 minutes)
 3) Bigtable (20 minutes)

We have allocated 15 minutes for set up and/or to address any other issues. 


This is the first Lab for the Big Data Systems course - Spring 2018.<br>
We are covering the following topics:
<li>Map-Reduce,</li>
<li>Spanner and</li>
<li>Bigtable</li>
<br>
The lab will have an in-class section as well as a homework section. <br>
You need to submit your in-class notebook before the end of the class. For the homework section, please refer to Canvas for the due date.


 

# <span style="color:#3665af">Section #2: Spanner </span><span style="font-size:15px">(Estimated time: 20 minutes) </span>

<hr>
In this section we will practice how to use Google's Spanner database. 
## Pre-reqs:
Your Google cloud account should be ready to deploy services.<br>

### Create a Spanner Instance.

- Create a spanner instace, and annotate the Instance ID. 
- We will use lab1-section2 as instance id in this lab; if you are using another id, you need to change in the connection settings. 
- For this test, configuring one node will suffice. 

### Installing the Python Client. 
We need to have the google-cloud library installed in our system.<br>
If you are using windows/mac, go to the anaconda navigator, then Environments. Click the arrow next to _base (root)_ and select Open terminal. <br>
Then execute the following pip command to install the client library. 

``` bash
$> pip install google-cloud
```

### Getting the Service Account File
As mentioned in the cloud setup instructions, you need to generate a token file so you will be able to connect to spanner. Refer to that document for help. <br>
Here is the summary of the steps. 
1. Go to APIS & Services
2. Go to Credentials
3. Select Create Credentials, and choose Service account key.
4. Select Compute Engine default service account and Key type JSON. Then Create. 
5. Pick the file that was automatically downloaded, and store it in your BSD directory. 


## Hands on...

In [ ]:
# Imports the Google Cloud Client Library.
from google.cloud import spanner


In [16]:
JSON_SERVICE_KEY = '/Users/crystinrodrick/Desktop/BigDataSystems-Spring2018-9b5e2d973ae2.json'

You can learn more about creating instances and using Spanner with Python at [Google Documentation](https://cloud.google.com/spanner/docs/getting-started/python/)


In [17]:
def explicit():
    ## Function to connect to spanner
    from google.cloud import storage

    # Explicitly use service account credentials by specifying the private key
    # file.
    storage_client = storage.Client.from_service_account_json(JSON_SERVICE_KEY)

    # Make an authenticated API request
    buckets = list(storage_client.list_buckets())
    print(buckets)

You can get help about authentication at [Google Documentation](https://cloud.google.com/docs/authentication/production#auth-cloud-explicit-python)


## Warm up
<hr>

In [20]:

# Instantiate a client.
spanner_client = spanner.Client.from_service_account_json(JSON_SERVICE_KEY)
#spanner_client = spanner.Client()

# Your Cloud Spanner instance ID.
instance_id = 'lab1-section2'
# Get a Cloud Spanner instance by ID.
instance = spanner_client.instance(instance_id)

# Your Cloud Spanner database ID.
database_id = 'lab-db'           # If you did not create the database already, createit using the cloud console

# Get a Cloud Spanner database by ID.
database = instance.database(database_id)

# Execute a simple SQL statement.
with database.snapshot() as snapshot:
    results = snapshot.execute_sql('SELECT current_date')

    for row in results:
        print(row)

[datetime.date(2018, 2, 28)]


/Users/crystinrodrick/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: DeprecationWarning: generator 'StreamedResultSet.__iter__' raised StopIteration


If you didn't recevie an error and the current date was displayed, it means that at this point we're connected to our Spanner database.<rb>

### Let's create a couple of tables
This may take a minute. If you use your google cloud dashboard, you can see the two new tables as you refresh the page.

In [33]:
operation = database.update_ddl(ddl_statements=[
    '''CREATE TABLE Singer (
        SingerId     INT64 NOT NULL,
        FirstName    STRING(1024),
        LastName     STRING(1024),
        SingerInfo   BYTES(MAX)
    ) PRIMARY KEY (SingerId)'''
    ,
    '''CREATE TABLE Album (
        SingerId     INT64 NOT NULL,
        AlbumId      INT64 NOT NULL,
        AlbumTitle   STRING(MAX)
    ) PRIMARY KEY (SingerId, AlbumId),
      INTERLEAVE IN PARENT Singer ON DELETE CASCADE'''
])

#operation = database.create()

print('Waiting for operation to complete...')
operation.result()

print('Created tables in database {} on instance {}'.format( database_id, instance_id))

Waiting for operation to complete...
Created tables in database lab-db on instance lab1-section2


### QUESTION:
**_In which line the command that creates the database is actually executed on Spanner? 
** 


On the command operation.result(). This is because the database gets structured during the  update_ddl command but is created when you call for the result. 

### Let's insert some data

In [34]:

"""Inserts sample data into the given database.
   The database and table must already exist.
"""

with database.batch() as batch:
    batch.insert(
        table='Singers',
        columns=('SingerId', 'FirstName', 'LastName',),
        values=[
            (1, u'Marc', u'Richards'),
            (2, u'Catalina', u'Smith'),
            (3, u'Alice', u'Trentor'),
            (4, u'Lea', u'Martin'),
            (5, u'David', u'Lomond')])
    batch.commit
    
    batch.insert(
        table='Albums',
        columns=('SingerId', 'AlbumId', 'AlbumTitle',),
        values=[
            (1, 1, u'Total Junk'),
            (1, 2, u'Go, Go, Go'),
            (2, 1, u'Green'),
            (2, 2, u'Forever Hold Your Peace'),
            (2, 3, u'Terrified')])
    batch.commit
print('Inserted data.')

Inserted data.


### QUESTION:
**_How many statements were executed in Spanner?_** Explain briefly. 

#### <span style="color:red">Two statements were executed in Spanner. The two being batch.commit. The reason being is that inserting all of the data moves the data into Spanner, once it commits then it is executed in Spanner. </span>

### Querying the database

In [49]:
"""Queries sample data from the database using SQL."""


with database.snapshot() as snapshot:
    results = snapshot.execute_sql(
        'SELECT SingerId, AlbumId, AlbumTitle FROM Albums')
    
    display(results)
    
    for row in results:
        display(row)
        #print(row[0])
        
        print('SingerId: {}, AlbumId: {}, AlbumTitle: {}'.format(*row))
        #print("SingerId: ",row[0],", AlbumId: ",row[1],", AlbumTitle: ",row[2], )
    #print(results)

[1, 1, 'Total Junk']

SingerId: 1, AlbumId: 1, AlbumTitle: Total Junk


[1, 2, 'Go, Go, Go']

SingerId: 1, AlbumId: 2, AlbumTitle: Go, Go, Go


[2, 1, 'Green']

...

[2, 2, 'Forever Hold Your Peace']

SingerId: 2, AlbumId: 2, AlbumTitle: Forever Hold Your Peace


[2, 3, 'Terrified']

SingerId: 2, AlbumId: 3, AlbumTitle: Terrified


/Users/crystinrodrick/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: generator 'StreamedResultSet.__iter__' raised StopIteration
  # Remove the CWD from sys.path while we load stuff.


### QUESTIONS:
1. **What kind of object is _results_?**
2. **What kind of object is _row_?**
3. **How is row being accessed?** _**Tip**_: take a look at the commented line.

#### <span style="color:red"> --- Answer HERE --- </span>
1. Results are all the singers with the albums they created. They come out as a list. 
2. Row is an instance of information pertaining to each singer and each album they created with the name of the album.
3. The row is being accessed by SingerID, Album, and AlbumTitle.

In [50]:
"""Reads sample data from the database."""

with database.snapshot() as snapshot:
    keyset = spanner.KeySet(all_=True)
    results = snapshot.read(
        table='Albums',
        columns=('SingerId', 'AlbumId', 'AlbumTitle',),
        keyset=keyset,)

    for row in results:
        display(row)
        print('SingerId: {}, AlbumId: {}, AlbumTitle: {}'.format(*row))
    #print(results)

[1, 1, 'Total Junk']

SingerId: 1, AlbumId: 1, AlbumTitle: Total Junk


[1, 2, 'Go, Go, Go']

SingerId: 1, AlbumId: 2, AlbumTitle: Go, Go, Go


[2, 1, 'Green']

...

[2, 2, 'Forever Hold Your Peace']

SingerId: 2, AlbumId: 2, AlbumTitle: Forever Hold Your Peace


[2, 3, 'Terrified']

SingerId: 2, AlbumId: 3, AlbumTitle: Terrified


/Users/crystinrodrick/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: generator 'StreamedResultSet.__iter__' raised StopIteration
  # Remove the CWD from sys.path while we load stuff.


### QUESTIONS:
**_What is the difference between the first and second part of the code?_**

#### <span style="color:red"> The difference between the first and second part of the code is that the first part uses a query whereas the second part just reads the data.  </span>


## Let's load some interesting data
<hr>

In [51]:
import pandas as pd
import datetime
from time import time

In [215]:

# Instantiate a client.
#spanner_client = spanner.Client()
spanner_client = spanner.Client.from_service_account_json(JSON_SERVICE_KEY)
# Your Cloud Spanner instance ID.
instance_id = 'lab1-section2'

# Get a Cloud Spanner instance by ID.
instance = spanner_client.instance(instance_id)

# Your Cloud Spanner database ID.
database_id = 'lab-db2'                             ## Create new Database

# Get a Cloud Spanner database by ID.
database = instance.database(database_id)

# Execute a simple SQL statement.
with database.snapshot() as snapshot:
    results = snapshot.execute_sql('SELECT current_date')

    for row in results:
        print(row)

[datetime.date(2018, 2, 28)]


/Users/crystinrodrick/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: generator 'StreamedResultSet.__iter__' raised StopIteration


In [176]:
start_time=time()
operation =database.update_ddl(ddl_statements=[
    """CREATE TABLE RadiationMeasurement
        (
           CapturedTime timestamp,
           Latitude float64,
           Longitude float64,
           Value float64,
           Unit String(2048),
           Location String(2048),
           DeviceID String(2048),
           MD5Sum String(2048),
           Height String(2048),
           Surface String(2048),
           Radiation String(2048),
           UploadedTime timestamp,
           LoaderID float64
        )PRIMARY KEY (CapturedTime,Latitude,Longitude,Location,UploadedTime,MD5Sum)"""
])

# operation = database.create()

print('Waiting for operation to complete...')
operation.result()

print('Created tables in database {} on instance {}'.format(
    database_id, instance_id))

end_time=time()

print ("run time:",end_time-start_time)

Waiting for operation to complete...
Created tables in database lab-db2 on instance lab1-section2
run time: 23.31500792503357


In [177]:
df_Measures        = pd.read_csv("/Users/crystinrodrick/Desktop/radiation.measurements.sample.csv")

In [216]:
df_Measures.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4999 entries, 0 to 4998
Data columns (total 13 columns):
Captured Time    4999 non-null object
Latitude         4999 non-null float64
Longitude        4999 non-null float64
Value            4999 non-null float64
Unit             4999 non-null object
Location Name    39 non-null object
Device ID        4338 non-null float64
MD5Sum           4999 non-null object
Height           3969 non-null float64
Surface          0 non-null float64
Radiation        0 non-null float64
Uploaded Time    4999 non-null object
Loader ID        661 non-null float64
dtypes: float64(8), object(5)
memory usage: 507.8+ KB


In [217]:
myData=[]
for index,row in df_Measures[0:5].iterrows():
    aDateTime = datetime.datetime.strptime(row["Captured Time"], '%Y-%m-%d %H:%M:%S')
    display('Row: {}, Timestamp: {} Lat/Lon:{}/{}'.format(index,aDateTime,row["Latitude"],row["Longitude"]))
    myData.append( (index,aDateTime)  )
myData

'Row: 0, Timestamp: 2020-02-03 17:00:00 Lat/Lon:37.507551666667/139.94117'

'Row: 1, Timestamp: 2020-02-03 11:00:00 Lat/Lon:37.505445/0.016666666666667'

'Row: 2, Timestamp: 2020-02-03 11:00:00 Lat/Lon:37.50725/139.94'

'Row: 3, Timestamp: 2020-02-01 03:00:00 Lat/Lon:34.066486666667/-118.89521666667'

'Row: 4, Timestamp: 2020-02-01 01:00:00 Lat/Lon:37.673233333333/140.06666666667'

[(0, datetime.datetime(2020, 2, 3, 17, 0)),
 (1, datetime.datetime(2020, 2, 3, 11, 0)),
 (2, datetime.datetime(2020, 2, 3, 11, 0)),
 (3, datetime.datetime(2020, 2, 1, 3, 0)),
 (4, datetime.datetime(2020, 2, 1, 1, 0))]

In [233]:
#     ****************************************************************
#     TO-DO: Write here a while/for to insert the Primary key data for the first 10 rows. 
#     ****************************************************************

# --Hints:
#     Adapt the code from the sample. 
#     Generate an array inserting tuples (you should use an array and the function append )
value = []
myData=[]
for index,row in df_Measures[0:10].iterrows():
    
    aDateTime = datetime.datetime.strptime(row["Captured Time"], '%Y-%m-%d %H:%M:%S')
    x = '{},{},{},{},{},{}'.format(aDateTime,row["Latitude"],row["Longitude"],row["Location Name"],row["MD5Sum"], row["Uploaded Time"])
    display('Row: {}, Timestamp: {} Lat/Lon:{}/{} Loc: {} MD5Sum: {} Uploaded Time: {}'.format(index,aDateTime,row["Latitude"],row["Longitude"],row["Location Name"],row["MD5Sum"], row["Uploaded Time"]))
    myData.append( (index,aDateTime) )
    value.append(row)
    
myData

with database.batch() as batch:
    batch.insert(
        table='RadiationMeasurement',
        columns=('Latitude', 'Longitude','Location', 'MD5Sum'), 
        values=[
            ( 37.507551666667, 139.94117, u'nan', u'6449bbf7ce3b30a8e05bc23a0bc40644'),
        (37.505445, 0.01666666666666, u'nan', u'a166df14f60b61095693684fc0f89c54' ),
        (37.50725, 139.94, u'nan', u'2fa8bccef282796bcdc297679c4db5b3'),
        (34.066486666667, -118.89521666667, u'nan', u'da79c21520d3ff3f5ed010a70f4a6d29'),
        (37.673233333333,140.06666666667,u'nan', u'6cdf740a2304a850185f74c0c4a4b878'),
        (37.674781666667,140.079895, u'nan', u'6666348e83cbf76e3508f473c302df77'),
        (37.517901666667, 139.92547833333, u'nan', u'5399edc68f223aba0e651fd5c8bdd933'),
        (37.737123333333, 140.72685333333, u'nan', u'42482c7028b1d843e2565b6c452057be'),
        (37.539609999999996, 140.11657666667, u'nan', u'e6f1f188b207a47d5482a12a1494d694'),
        (37.75588, 140.70341166667, u'nan', u'29ab5e66f6bcd461f61db388616c3f39')])
            
    batch.commit
print('Inserted data.')


'Row: 0, Timestamp: 2020-02-03 17:00:00 Lat/Lon:37.507551666667/139.94117 Loc: nan MD5Sum: 6449bbf7ce3b30a8e05bc23a0bc40644 Uploaded Time: 2020-02-03 17:00:00'

'Row: 1, Timestamp: 2020-02-03 11:00:00 Lat/Lon:37.505445/0.016666666666667 Loc: nan MD5Sum: a166df14f60b61095693684fc0f89c54 Uploaded Time: 2020-02-03 11:00:00'

'Row: 2, Timestamp: 2020-02-03 11:00:00 Lat/Lon:37.50725/139.94 Loc: nan MD5Sum: 2fa8bccef282796bcdc297679c4db5b3 Uploaded Time: 2020-02-03 11:00:00'

'Row: 3, Timestamp: 2020-02-01 03:00:00 Lat/Lon:34.066486666667/-118.89521666667 Loc: nan MD5Sum: da79c21520d3ff3f5ed010a70f4a6d29 Uploaded Time: 2020-02-01 03:00:00'

'Row: 4, Timestamp: 2020-02-01 01:00:00 Lat/Lon:37.673233333333/140.06666666667 Loc: nan MD5Sum: 6cdf740a2304a850185f74c0c4a4b878 Uploaded Time: 2020-02-01 01:00:00'

...

'Row: 6, Timestamp: 2020-01-12 19:00:00 Lat/Lon:37.517901666667/139.92547833333 Loc: nan MD5Sum: 5399edc68f223aba0e651fd5c8bdd933 Uploaded Time: 2020-01-12 19:00:00'

'Row: 7, Timestamp: 2020-01-12 19:00:00 Lat/Lon:37.737123333333/140.72685333333 Loc: nan MD5Sum: 42482c7028b1d843e2565b6c452057be Uploaded Time: 2020-01-12 19:00:00'

'Row: 8, Timestamp: 2020-01-12 19:00:00 Lat/Lon:37.539609999999996/140.11657666667 Loc: nan MD5Sum: e6f1f188b207a47d5482a12a1494d694 Uploaded Time: 2020-01-12 19:00:00'

'Row: 9, Timestamp: 2020-01-12 19:00:00 Lat/Lon:37.75588/140.70341166667 Loc: nan MD5Sum: 29ab5e66f6bcd461f61db388616c3f39 Uploaded Time: 2020-01-12 19:00:00'

Inserted data.


# <span style="color:#5DB664">DELETE YOUR SPANNER INSTANCE AS WHEN YOU FINNISHED </span>

<hr><BR>

# <span style="color:RED">Homework: </span> Yelp Dataset
<hr>
Similar to the example above, we are going to be using the Yelp competition dataset available [here](https://www.kaggle.com/yelp-dataset/yelp-dataset/data). <br>
You should:
1. Download the dataset to your machine.
2. Create a new database.
3. Create the tables necessary, with the proper interleaving configuration to store the business and reviews.
4. Load the yelp_business dataset.
5. Load 10% of the yelp_review dataset.

Hints:
- You probably want to do several batchs. You can do that by using the **[start:end]** operator for the dataframes. 
- You may need to review the documentation for the [Spanner SQL](https://cloud.google.com/spanner/docs/query-syntax)

## Questions:
1. Report the time for loading each dataset.
2. Compare the time to load the rows for the business and the reviews datasets.
3. List the top 10 better/most rated businesses in Colorado. To simplify this, let's assume that the best rated business is the one with the higher sum of ratings. 



In [23]:
# Imports the Google Cloud Client Library.
from google.cloud import spanner

<hr style="border: 3px double navy;" ><br><br><br>

In [24]:
JSON_SERVICE_KEY = '/Users/crystinrodrick/Desktop/BigDataSystems-Spring2018-35b206318c5f.json'

In [25]:
def explicit():
    ## Function to connect to spanner
    from google.cloud import storage

    # Explicitly use service account credentials by specifying the private key
    # file.
    storage_client = storage.Client.from_service_account_json(JSON_SERVICE_KEY)

    # Make an authenticated API request
    buckets = list(storage_client.list_buckets())
    print(buckets)

In [26]:
# Instantiate a client.
spanner_client = spanner.Client.from_service_account_json(JSON_SERVICE_KEY)
#spanner_client = spanner.Client()

# Your Cloud Spanner instance ID.
instance_id = 'lab1-section2'

# Get a Cloud Spanner instance by ID.
instance = spanner_client.instance(instance_id)

# Your Cloud Spanner database ID.
database_id = 'lab-db'           # If you did not create the database already, createit using the cloud console

# Get a Cloud Spanner database by ID.
database = instance.database(database_id)

# Execute a simple SQL statement.
with database.snapshot() as snapshot:
    results = snapshot.execute_sql('SELECT current_date')

    for row in results:
        print(row)

[datetime.date(2018, 3, 1)]


/Users/crystinrodrick/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: generator 'StreamedResultSet.__iter__' raised StopIteration


In [92]:
operation = database.update_ddl(ddl_statements=[
    '''CREATE TABLE BusiNess (
        business_id  STRING(1024),
        name         STRING(1024),
        neighborhood STRING(1024),
        address      STRING(1024),
        city         STRING(1024),
        state        STRING(1024),
        postal_code  STRING(1024),
        latitude     float64,
        longitude    float64,
        stars        float64,
        review_count INT64,
        is_open      INT64,
        categories   STRING(2048)
    ) PRIMARY KEY (business_id)'''
    ,
    '''CREATE TABLE ReView (
        review_id     STRING(1024),
        user_id       STRING(1024),
        business_id   STRING(1024),
        stars         INT64,
        date          STRING(1024),
        text          STRING(2048),
        useful        INT64,
        funny         INT64,
        cool          INT64
    ) PRIMARY KEY (business_id, review_id),
      INTERLEAVE IN PARENT BusiNess ON DELETE CASCADE'''
])

#operation = database.create()

print('Waiting for operation to complete...')
operation.result()

print('Created tables in database {} on instance {}'.format( database_id, instance_id))

Waiting for operation to complete...
Created tables in database lab-db on instance lab1-section2


In [93]:
import pandas as pd
df1 = pd.read_csv("/Users/crystinrodrick/Desktop/yelp_review.csv", encoding ='utf-8', nrows=500000) #read_csv(..., skiprows=1000000, nrows=999999)

In [94]:
from time import time
start_time=time()
df1.info()

print('Created tables in database {} on instance {}'.format(
    database_id, instance_id))

end_time=time()

print ("run time:", end_time-start_time)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 9 columns):
review_id      500000 non-null object
user_id        500000 non-null object
business_id    500000 non-null object
stars          500000 non-null int64
date           500000 non-null object
text           500000 non-null object
useful         500000 non-null int64
funny          500000 non-null int64
cool           500000 non-null int64
dtypes: int64(4), object(5)
memory usage: 34.3+ MB
Created tables in database lab-db on instance lab1-section2
run time: 0.24693894386291504


In [95]:
df2 = pd.read_csv("/Users/crystinrodrick/Desktop/yelp_business.csv", encoding = 'utf-8')

In [96]:
start_time=time()
df2.info()
print('Created tables in database {} on instance {}'.format(
    database_id, instance_id))

end_time=time()

print ("run time:",end_time-start_time)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174567 entries, 0 to 174566
Data columns (total 13 columns):
business_id     174567 non-null object
name            174567 non-null object
neighborhood    68015 non-null object
address         174567 non-null object
city            174566 non-null object
state           174566 non-null object
postal_code     173944 non-null object
latitude        174566 non-null float64
longitude       174566 non-null float64
stars           174567 non-null float64
review_count    174567 non-null int64
is_open         174567 non-null int64
categories      174567 non-null object
dtypes: float64(3), int64(2), object(8)
memory usage: 17.3+ MB
Created tables in database lab-db on instance lab1-section2
run time: 0.11225199699401855


In [97]:
batch_size = 1000
batch_start = 0
start_time=time()
counter = 1
#values = []
for ind, row in df2[batch_start:batch_size].iterrows():
    with database.batch() as batch: 
        
        batch.insert(
            table='BusiNess',
            columns=('business_id', 'name','neighborhood', 'address', 'city', 'state', 'postal_code', 'latitude', 
                'longitude', 'stars', 'review_count', 'is_open', 'categories'),
            values = [(row['business_id'], 
                    row['name'],
                    row['neighborhood'], 
                    row['address'], 
                    row['city'], 
                    row['state'], 
                    row['postal_code'], 
                    row['latitude'], 
                    row['longitude'], 
                    row['stars'], 
                    row['review_count'], 
                    row['is_open'], 
                      row['categories'])]
        )          
        batch.commit    
    print("Inserted batch: ", counter)
    counter +=1
    batch_size += 1000
    batch_start += 1000
    
        
end_time=time()

print ("run time:",end_time-start_time)

Inserted batch:  1
Inserted batch:  2
Inserted batch:  3
Inserted batch:  4
Inserted batch:  5
Inserted batch:  6
Inserted batch:  7
Inserted batch:  8
Inserted batch:  9
Inserted batch:  10
Inserted batch:  11
Inserted batch:  12
Inserted batch:  13
Inserted batch:  14
Inserted batch:  15
Inserted batch:  16
Inserted batch:  17
Inserted batch:  18
Inserted batch:  19
Inserted batch:  20
Inserted batch:  21
Inserted batch:  22
Inserted batch:  23
Inserted batch:  24
Inserted batch:  25
Inserted batch:  26
Inserted batch:  27
Inserted batch:  28
Inserted batch:  29
Inserted batch:  30
Inserted batch:  31
Inserted batch:  32
Inserted batch:  33
Inserted batch:  34
Inserted batch:  35
Inserted batch:  36
Inserted batch:  37
Inserted batch:  38
Inserted batch:  39
Inserted batch:  40
Inserted batch:  41
Inserted batch:  42
Inserted batch:  43
Inserted batch:  44
Inserted batch:  45
Inserted batch:  46
Inserted batch:  47
Inserted batch:  48
Inserted batch:  49
Inserted batch:  50
Inserted 

Inserted batch:  400
Inserted batch:  401
Inserted batch:  402
Inserted batch:  403
Inserted batch:  404
Inserted batch:  405
Inserted batch:  406
Inserted batch:  407
Inserted batch:  408
Inserted batch:  409
Inserted batch:  410
Inserted batch:  411
Inserted batch:  412
Inserted batch:  413
Inserted batch:  414
Inserted batch:  415
Inserted batch:  416
Inserted batch:  417
Inserted batch:  418
Inserted batch:  419
Inserted batch:  420
Inserted batch:  421
Inserted batch:  422
Inserted batch:  423
Inserted batch:  424
Inserted batch:  425
Inserted batch:  426
Inserted batch:  427
Inserted batch:  428
Inserted batch:  429
Inserted batch:  430
Inserted batch:  431
Inserted batch:  432
Inserted batch:  433
Inserted batch:  434
Inserted batch:  435
Inserted batch:  436
Inserted batch:  437
Inserted batch:  438
Inserted batch:  439
Inserted batch:  440
Inserted batch:  441
Inserted batch:  442
Inserted batch:  443
Inserted batch:  444
Inserted batch:  445
Inserted batch:  446
Inserted batc

Inserted batch:  792
Inserted batch:  793
Inserted batch:  794
Inserted batch:  795
Inserted batch:  796
Inserted batch:  797
Inserted batch:  798
Inserted batch:  799
Inserted batch:  800
Inserted batch:  801
Inserted batch:  802
Inserted batch:  803
Inserted batch:  804
Inserted batch:  805
Inserted batch:  806
Inserted batch:  807
Inserted batch:  808
Inserted batch:  809
Inserted batch:  810
Inserted batch:  811
Inserted batch:  812
Inserted batch:  813
Inserted batch:  814
Inserted batch:  815
Inserted batch:  816
Inserted batch:  817
Inserted batch:  818
Inserted batch:  819
Inserted batch:  820
Inserted batch:  821
Inserted batch:  822
Inserted batch:  823
Inserted batch:  824
Inserted batch:  825
Inserted batch:  826
Inserted batch:  827
Inserted batch:  828
Inserted batch:  829
Inserted batch:  830
Inserted batch:  831
Inserted batch:  832
Inserted batch:  833
Inserted batch:  834
Inserted batch:  835
Inserted batch:  836
Inserted batch:  837
Inserted batch:  838
Inserted batc

In [ ]:
batch_size = 1000
batch_start = 0
start_time=time()
counter = 1

for ind, row in df1[batch_start:batch_size].iterrows():
    with database.batch() as batch:
    
        
        batch.insert(
            table='ReView',
                    
            columns=('review_id', 'user_id','business_id', 'stars', 'date', 'text', 'useful', 'funny', 
                'cool'),
            values = [(row['review_id'],
                        row['user_id'],
                        row['business_id'],  
                        row['stars'],      
                        row['date'],      
                        row['text'],        
                        row['useful'],       
                        row['funny'],       
                        row['cool'] )
                     ])
        
            
        batch.commit
    print("Inserted batch: ", counter)
    counter +=1
    batch_size += 1000
    batch_start += 1000

end_time=time()

print ("run time:",end_time-start_time)

In [111]:
with database.snapshot() as snapshot:
    results = snapshot.execute_sql(
        'SELECT state, stars, business_id FROM BusiNess WHERE state = "CO" ORDER BY Stars DESC')
    display(results)
    for row in results:
        display(row)


['CO', 3.0, 'RBOXMNb_gzdmy1CRPxv_4A']

/Users/crystinrodrick/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: generator 'StreamedResultSet.__iter__' raised StopIteration
  """
